<a href="https://colab.research.google.com/github/marcocmcmelo/projetos_python/blob/main/previsao_score_credito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto Python IA: Inteligência Artificial e Previsões

### Case: Score de Crédito dos Clientes

Você foi contratado por um banco para conseguir definir o score de crédito dos clientes. Você precisa analisar todos os clientes do banco e, com base nessa análise, criar um modelo que consiga ler as informações do cliente e dizer automaticamente o score de crédito dele: Ruim, Ok, Bom.

In [1]:
# passo a passo
# passo 0 - entender a necessidade da empresa
# passo 1 - importar a base de dados
# passo 2 - preparar a base de dados para IA
# passo 3 - treinar a IA - criar o modelo nota de crédito: boa, ok, ruim.

!pip install scikit-learn

In [43]:
# LabelEncoder - tarsnforma as colunas de texto profissões, cinetista, mecânico, etc em numeros 0, 1 , 2 ...
# isso porque nesse caso para treinar (no processo de aprendizado) a IA é só em campos numericos
from sklearn.preprocessing import LabelEncoder

# separa os dados de treino e de teste
from sklearn.model_selection import train_test_split

# gera através do modelo de árvore de descisão
from sklearn.ensemble import RandomForestClassifier

# gera através do modelo KNN (nearest neighbors - vizinhos mais próximos)
from sklearn.neighbors import KNeighborsClassifier

# calcula as previsões
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score

# gera através do modelo de regressão linear
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

import pandas as pd
import numpy as np

In [44]:
# importando a base de dados
tabela = pd.read_csv('clientes.csv')
display(tabela)

,id_cliente,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,...,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,score_credito,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,3392,1,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,265.0,21.465380,alto_gasto_pagamento_baixos,312.494089,Good,1,1,1,1,0
1,3392,2,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,266.0,21.465380,baixo_gasto_pagamento_alto,284.629162,Good,1,1,1,1,0
2,3392,3,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,267.0,21.465380,baixo_gasto_pagamento_medio,331.209863,Good,1,1,1,1,0
3,3392,4,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,5.0,...,268.0,21.465380,baixo_gasto_pagamento_baixo,223.451310,Good,1,1,1,1,0
4,3392,5,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,6.0,...,269.0,21.465380,alto_gasto_pagamento_medio,341.489231,Good,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,37932,4,25.0,mecanico,39628.99,4.0,6.0,7.0,2.0,23.0,...,378.0,24.028477,alto_gasto_pagamento_alto,479.866228,Poor,1,0,0,0,1
99996,37932,5,25.0,mecanico,39628.99,4.0,6.0,7.0,2.0,18.0,...,379.0,24.028477,alto_gasto_pagamento_medio,496.651610,Poor,1,0,0,0,1
99997,37932,6,25.0,mecanico,39628.99,4.0,6.0,7.0,2.0,27.0,...,380.0,24.028477,alto_gasto_pagamento_alto,516.809083,Poor,1,0,0,0,1
99998,37932,7,25.0,mecanico,39628.99,4.0,6.0,7.0,2.0,20.0,...,381.0,24.028477,baixo_gasto_pagamento_alto,319.164979,Standard,1,0,0,0,1


In [45]:
print(tabela.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  object 
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  object 
 13  divida_total              100000 non-null  fl

In [46]:
print(tabela.columns)

Index(['id_cliente', 'mes', 'idade', 'profissao', 'salario_anual',
       'num_contas', 'num_cartoes', 'juros_emprestimo', 'num_emprestimos',
       'dias_atraso', 'num_pagamentos_atrasados', 'num_verificacoes_credito',
       'mix_credito', 'divida_total', 'taxa_uso_credito',
       'idade_historico_credito', 'investimento_mensal',
       'comportamento_pagamento', 'saldo_final_mes', 'score_credito',
       'emprestimo_carro', 'emprestimo_casa', 'emprestimo_pessoal',
       'emprestimo_credito', 'emprestimo_estudantil'],
      dtype='object')


In [47]:
# transformar as colunas de txto (profissao, mix_credito, comportamento_pagamento)
# em numerico ( score_credito não precisa porque pe a coluna que vamos prever)
codificador_profissao = LabelEncoder()
tabela['profissao'] = codificador_profissao.fit_transform(tabela['profissao'])

codificador_mixcredito = LabelEncoder()
tabela['mix_credito'] = codificador_mixcredito.fit_transform(tabela['mix_credito'])

codificador_pgto = LabelEncoder()
tabela['comportamento_pagamento'] = codificador_pgto.fit_transform(tabela['comportamento_pagamento'])

display(tabela.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  int64  
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  int64  
 13  divida_total              100000 non-null  fl

None

In [ ]:
# o codigo acima poderia ser feito dessa forma

#   codificador = LabelEncoder()

#   for coluna in tabela.columns:
#     if tabela[coluna].dtype == "object" and coluna != "score_credito":
#       tabela[coluna] = codificador.fit_transform(tabela[coluna])


In [48]:
#deve-se escolher a coluna que se quer usar para treinar o modelo
# y será a coluna que o  modelo irá calcular
# x são as outras colunas que serão usadas para prevero score de credito
# não será usadas as colunas score_credito que é a cluna a se prevista
# não será usada a coluna id_cliente que é uma coluna de identificaçaõ de
# codigo cliente e ela não tem relevancia para essa previsão

x = tabela.drop(['score_credito', 'id_cliente'], axis=1)
y = tabela["score_credito"]

# separam-se os dados de treino e de teste.
# treino é para o modelo aprender e teste é usado para ver se o modelo aprendeu corretamente
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state=1)

In [49]:
# modelo árvore de descisão
modelo_arvore = RandomForestClassifier()

# modelo KNN (naerest neighbors - vizinhos mais próximos)
modelo_knn = KNeighborsClassifier()

# Criando o modelo de regressão linear
modelo_revisoes = LogisticRegression()


Para essa base de dados o modelo de revisão linear não se mostrou eficiente ficando com o pior %, o que melhor apresentou resultado foi o modelo de Árvore.

1.  Árvor de descisão: 82,48%
2. KNN: 73,24%
3. Revisão Linear: 54,52%




In [50]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np

# Convertendo os valores categóricos de y_treino e y_teste
label_encoder = LabelEncoder()
y_treino = label_encoder.fit_transform(y_treino)
y_teste = label_encoder.transform(y_teste)

# Treinando os modelos
modelo_arvore.fit(x_treino, y_treino)
modelo_knn.fit(x_treino, y_treino)
modelo_revisoes.fit(x_treino, y_treino)

# Fazendo previsões
previsao_arvore = modelo_arvore.predict(x_teste)
previsao_knn = modelo_knn.predict(x_teste)
previsoes = modelo_revisoes.predict(x_teste)

# Ajustando previsões contínuas (caso o modelo seja de regressão)
if hasattr(modelo_revisoes, "predict"):
    previsoes = np.round(previsoes).astype(int)

# Calculando acurácia
print(f'Acurácia Árvore de Decisão: {accuracy_score(y_teste, previsao_arvore) * 100:.2f}%')
print(f'Acurácia KNN: {accuracy_score(y_teste, previsao_knn) * 100:.2f}%')
print(f'Acurácia Modelo Revisões: {accuracy_score(y_teste, previsoes) * 100:.2f}%')


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Acurácia Árvore de Decisão: 82.48%
Acurácia KNN: 73.24%
Acurácia Modelo Revisões: 54.52%


In [21]:
# Fazendo a contagem da coluna score_credito
contagem_scores = tabela["score_credito"].value_counts()

# Calculando a proporção de "Standard" em relação ao total
acuracia_chute = contagem_scores['Standard'] / sum(contagem_scores)

# Exibindo o resultado em porcentagem
print(f"Acurácia do chute 'Standard': {acuracia_chute * 100:.2f}%")


Acurácia do chute 'Standard': 53.17%
